<a href="https://colab.research.google.com/github/mercadoerik1031/snn-sound-localization/blob/new_approach/snn_sound_localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SNN Sounnd Localization**



---



In [9]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# pip Installs

# Config

In [10]:
config = {
    # Paths
    "metadata_path": r"/content/drive/My Drive/Colab Notebooks/Masters Project/metadata.parquet",
    "ambisonic_path": r"/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/ambisonics_sample",
    "noise_path": r"/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/noise_ambisonics_sample",

    # Device
    #"device": "cuda" if torch.cuda.is_available() else "cpu",

    # Audio Info
    "sr":16_000,
}

# Imports

In [11]:
import librosa
import numpy as np

import scipy

In [12]:
def load_audio(ambisonics_path, noise_path=None, sr=config["sr"]):
  """
  Load Audio
  """
  ambisonic_audio, _ = librosa.load(ambisonics_path, sr=sr, mono=False)

  if not noise_path:
    return ambisonic_audio

  noise_audio, _ = librosa.load(noise_path, sr=sr, mono=False)

  return ambisonic_audio + noise_audio

In [13]:
def extract_ITD(audio_data, sr=config["sr"]):
  """
  Interaural Time Difference (ITD) extraction
  """

  W = audio_data[0,:]
  X = audio_data[1,:]

  correlation = np.correlate(W, X, "full")

  time_delay = np.argmax(correlation) - len(W)

  # Convert time delay in samples to milliseconds
  itd = time_delay / sr * 100

  return itd

In [14]:
def extract_ILD(audio_data, sr=config["sr"]):
  """
  Interaural Level Difference (ILD) extraction
  """

  Y = audio_data[2,:]
  Z = audio_data[3,:]

  intensity_Y = np.square(Y)
  intensity_Z = np.square(Z)

  ild = intensity_Y - intensity_Z

  return ild